In [2]:
from importlib import reload
import cvolume.utils, cvolume.series, cvolume.localpoly, cvolume.stable_graphs, cvolume.cvolume
reload(cvolume); reload(cvolume.utils); reload(cvolume.series); reload(cvolume.localpoly)
reload(cvolume.stable_graphs), reload(cvolume.cvolume)
import cvolume; reload(cvolume);

In [3]:
from sage.misc.sageinspect import sage_getdoc
from sage.symbolic.constants import pi
from sage.rings.integer_ring import ZZ
from cvolume import *
from cvolume.series import Fs
from cvolume.cvolume import volumes_dict, cvolumes_dict, conjvolumes_dict
from cvolume.utils import stratum2print as s2p

In [5]:
!pip install ./prettytable;
from prettytable import PrettyTable
import pickle

pip is configured with locations that require TLS/SSL, however the ssl module in Python is not available.
Processing ./prettytable
  Found existing installation: prettytable 1.0.1
    Uninstalling prettytable-1.0.1:
      Successfully uninstalled prettytable-1.0.1
  Running setup.py install for prettytable ... done
pip is configured with locations that require TLS/SSL, however the ssl module in Python is not available.
Could not fetch URL https://pypi.org/simple/pip/: There was a problem confirming the ssl certificate: HTTPSConnectionPool(host='pypi.org', port=443): Max retries exceeded with url: /simple/pip/ (Caused by SSLError("Can't connect to HTTPS URL because the SSL module is not available.")) - skipping


In [6]:
def show_strata(higher_part, num):
    higher_part = sorted(higher_part,reverse=True)
    kappa = sum(higher_part)
    l = len(higher_part)
    s = kappa//4
    eps = 4*(s+1)-kappa
    f = 2*(s+1)+eps+l
    ans = []
    min_d = max(2-f%2,f+6*(-s-2))
    count = 0
    for d in range(min_d,min_d+2*num,2):
        for k in range(-s-2,(d-f)//6+1):
            n = (d-f)//2-3*k
            m = n + eps + 4*k
            g = (kappa+m-n+4)//4
            stratum = higher_part + [1]*m + [-1]*n
            if m >= 0 and n >= 0 and (kappa+m-n)%4==0 and g >= 0:
                ans.append((d,g,stratum,s2p(stratum)))
                count += 1
            if count >= num: break
        if count >= num: break
    return sorted(ans,key = lambda x : (x[1],x[0],-x[2].count(-1)))

def get_local_dicts():
    with open('cvolume/volumes.pkl','rb') as d:
        local_volumes = pickle.load(d)
    with open('cvolume/cvolumes.pkl','rb') as d:
        local_cvolumes = pickle.load(d)
    with open('cvolume/ovolumes.pkl','rb') as d:
        local_ovolumes = pickle.load(d)
    with open('cvolume/conjvolumes.pkl','rb') as d:
        local_conjvolumes = pickle.load(d)
    return local_volumes,local_cvolumes,local_ovolumes,local_conjvolumes

def update_local_dicts(local_volumes,local_cvolumes,local_ovolumes,local_conjvolumes):
    for stratum in set(volumes_dict.keys())-set(local_volumes.keys()):
        local_volumes[stratum] = volumes_dict[stratum][0]
    for stratum in set(cvolumes_dict.keys())-set(local_cvolumes.keys()):
        local_cvolumes[stratum] = cvolumes_dict[stratum]
    for stratum in set(conjvolumes_dict.keys())-set(local_conjvolumes.keys()):
        local_conjvolumes[stratum] = conjvolumes_dict[stratum]
    with open('cvolume/volumes.pkl','wb') as d:
        pickle.dump(local_volumes,d)
    with open('cvolume/cvolumes.pkl','wb') as d:
        pickle.dump(local_cvolumes,d)
    with open('cvolume/ovolumes.pkl','wb') as d:
        pickle.dump(local_ovolumes,d)
    with open('cvolume/conjvolumes.pkl','wb') as d:
        pickle.dump(local_conjvolumes,d)
        
def record_volumes(higher_part,table,num,mode='default'):
    strata = show_strata(higher_part,10*num)
    count = 0
    local_volumes,local_cvolumes,local_ovolumes,local_conjvolumes = get_local_dicts()
    for d,g,stratum,pretty_stratum in strata:
        if stratum.count(-1) <= g:
            stratum = sorted(stratum,reverse=True)
            a_volume = asymptotic_volume(stratum)
            if tuple(stratum) in local_cvolumes:
                c_volume = local_cvolumes[tuple(stratum)]
            else:
                c_volume = completed_volume(stratum,verbose=True)
            if tuple(stratum) in local_ovolumes:
                ov_volume = local_ovolumes[tuple(stratum)]
            else:
                ov_volume = completed_volume(stratum,one_vertex=True)
                local_ovolumes[tuple(stratum)] = ov_volume
            if tuple(stratum) in local_volumes:
                volume = local_volumes[tuple(stratum)]
            elif mode == 'conjecture' and tuple(stratum) in local_conjvolumes:
                volume = local_conjvolumes[tuple(stratum)]
            else:
                volume = MV_volume(stratum,mode=mode)                
            numbers = [volume,c_volume,a_volume,volume/a_volume,c_volume/a_volume,volume/c_volume,ov_volume/c_volume]
            row = [g,d,pretty_stratum] + [f'{float(number):.3f}' for number in numbers]
            table.add_row(row)
            count += 1
            if count >= num:
                break
    update_local_dicts(local_volumes,local_cvolumes,local_ovolumes,local_conjvolumes)

In [ ]:
print('Notation:')
print('Vol     Masur-Veech volume'.ljust(40),'g    genus')
print('cVol    completed volume'.ljust(40),'d    dimension')
print('aVol    conjectural asymptotic volume')
print('oVol    one-vertex graphs contribution')

print('Strata [3,1,...]:')
table = PrettyTable(['g','dim','stratum','Vol','cVol','aVol','Vol/aVol','cVol/aVol','Vol/cVol','oVol/cVol'])
record_volumes([3],table,11)
print(table)

print('Strata [5,1,...]:')
table = PrettyTable(['g','dim','stratum','Vol','cVol','aVol','Vol/aVol','cVol/aVol','Vol/cVol','oVol/cVol'])
record_volumes([5],table,10)
print(table)

print('Strata [5,3,1,...]:')
table = PrettyTable(['g','dim','stratum','Vol','cVol','aVol','Vol/aVol','cVol/aVol','Vol/cVol','oVol/cVol'])
record_volumes([5,3],table,7,mode='conjecture')
print(table)

Notation:
Vol     Masur-Veech volume               g    genus
cVol    completed volume                 d    dimension
aVol    conjectural asymptotic volume
oVol    one-vertex graphs contribution
Strata [3,1,...]:
Computing completed volume of stratum [3,1^12,-1^3]...
Generated 14  codimension 1 graphs in 0.029 s
Generated 109 codimension 2 graphs in 0.307 s
Generated 599 codimension 3 graphs in 2.126 s
Generated 2497 codimension 4 graphs in 9.621 s
Generated 7940 codimension 5 graphs in 40.248 s
Generated 19141 codimension 6 graphs in 1 min 56.981 s
Generated 34182 codimension 7 graphs in 4 min 24.442 s
Generated 43629 codimension 8 graphs in 5 min 41.697 s
Generated 37557 codimension 9 graphs in 6 min 8.269 s
Generated 19554 codimension 10 graphs in 4 min 24.579 s
Generated 4718 codimension 11 graphs in 1 min 27.551 s
The total number of stable graphs for stratum [3,1^12,-1^3] is: 169940. Generated in: 24 min 56.031 s
Computed contribution of 120660/169940 graphs. Time elapsed: 73 min

In [8]:
Fs.verbose=True